<a href="https://colab.research.google.com/github/elka97/DataAnalysisAlgoritms/blob/main/homework_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [159]:
import numpy as np

In [160]:
def standard_scaler(x):
    res = (x - x.mean()) / x.std()
    return res

def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res 

def calc_logloss(y, y_pred):
    err = np.mean(- y * np.log(y_pred) - (1.0 - y) * np.log(1.0 - y_pred))
    return err 

1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log (как вариант - использовать np.clip или np.where).

In [161]:
def calc_logloss_no_zeros(y, y_pred, log_adjust=1e-1):        
    ymin=0+log_adjust
    ymax=1-log_adjust
    _y = np.clip(y, a_min = ymin, a_max =ymax) 
    _y_pred = np.clip(y_pred, a_min = ymin, a_max = ymax)
    err = np.mean(- y * np.log(_y_pred) - (1.0 - _y) * np.log(1.0 - _y_pred))
    return err

def eval_LR_model_no_zeros(X, y, iterations, alpha=1e-4, log_adjust=1e-1):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w) # log(p/(1-p))
        y_pred = sigmoid(z) # p [0, 1]
        err = calc_logloss_no_zeros(y, y_pred, log_adjust)
        w -= alpha * (1/n * np.dot(X.T, (y_pred - y)))
        if i % (iterations / 10) == 0:
            print(i, w, err)
    return w

In [180]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],  # квалификация репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # поступил или нет ученик на специальность Математика

X_st = X.copy()
X_st[:, 2] = standard_scaler(X[:, 2])

log_adjust = 1e-8
w = eval_LR_model_no_zeros(X_st, y, 2000, 1e-2, log_adjust) 
w

200 [ 0.14805533 -0.69317972  0.77965488  1.21287192] 0.5687278091774088
400 [-0.00887935 -0.68415387  0.90713736  1.22443854] 0.5476800304816031
600 [-0.1239972  -0.69202069  0.98318257  1.26924759] 0.5369841892969819
800 [-0.21681092 -0.7078635   1.03062761  1.32973537] 0.5295551533064984
1000 [-0.29764618 -0.72670545  1.061634    1.39666497] 0.5233765362660872
1200 [-0.37188064 -0.7461597   1.08260067  1.46531853] 0.5178498879052098
1400 [-0.44230332 -0.76510554  1.09696619  1.53338645] 0.5127677872523864
1600 [-0.51036921 -0.783047    1.10664485  1.59979796] 0.5080360429391497
1800 [-0.57684595 -0.79979671  1.11275552  1.66410902] 0.503597787278171
2000 [-0.64214517 -0.81531815  1.11599369  1.72618787] 0.49941172112416715


array([-0.64214517, -0.81531815,  1.11599369,  1.72618787])



---



2. Подберите аргументы функции eval_LR_model для логистической регрессии таким образом, чтобы log loss был минимальным. Покажите влияние гиперпараметров на ошибку алгоритма (оптимально здесь использовать критерий остановки обучения).

In [181]:
def eval_LR_model_early_stop(X, y, iterations, alpha=1e-4, min_weight_dist=1e-5, debug=False):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]    
    tmp = np.inf
    errors=[]
    for i in range(1, iterations + 1):
        z = np.dot(X, w) # log(p/(1-p))
        y_pred = sigmoid(z) # p [0, 1]
        err = calc_logloss(y, y_pred)
        errors.append(err)
        prev_w = w
        w -= alpha * (1/n * np.dot(X.T, (y_pred - y)))        
        if debug and i % (iterations / 10) == 0:
            print(i, w, err)
        if abs(tmp-err) <= min_weight_dist:
            # print(f"{i} iterations need")
            print(i, w, err, f" Stopped on {i} iteration")
            break
        tmp=err
    return w, errors, i

In [182]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],  # квалификация репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # поступил или нет ученик на специальность Математика

X_st = X.copy()
X_st[:, 2] = standard_scaler(X[:, 2])

best = {"weights": np.inf, "iterations": np.inf, "error": np.inf, "alpha":np.inf, "early_stop_iteration":np.inf}

# find optimal learning rate
for iterations in range(1000, 10000, 1000):
  for alpha in np.logspace(start=-3, stop=-1, num=5):
      # print(iterations, alpha)      
      w, errs, early_stop_iter = eval_LR_model_early_stop(X_st, y, iterations, alpha, debug=False)
      if errs[-1] < best.get("error"):
          best = {"weights": w, "iterations": iterations, "error": errs[-1], "alpha": alpha, "early_stop_iteration":early_stop_iter}

print("\nBest paramters:")
best

2725 [-0.24263494 -0.71350201  1.04144098  1.34998906] 0.5275400371619717  Stopped on 2725 iteration
3017 [ 0.06146141 -0.68603187  0.85260956  1.21305757] 0.5558187108126926  Stopped on 3017 iteration
2725 [-0.24263494 -0.71350201  1.04144098  1.34998906] 0.5275400371619717  Stopped on 2725 iteration
3017 [ 0.06146141 -0.68603187  0.85260956  1.21305757] 0.5558187108126926  Stopped on 3017 iteration
2725 [-0.24263494 -0.71350201  1.04144098  1.34998906] 0.5275400371619717  Stopped on 2725 iteration
3017 [ 0.06146141 -0.68603187  0.85260956  1.21305757] 0.5558187108126926  Stopped on 3017 iteration
2725 [-0.24263494 -0.71350201  1.04144098  1.34998906] 0.5275400371619717  Stopped on 2725 iteration
5853 [-8.53579974 -1.24563049 -1.51151549  7.33020294] 0.28384397239036285  Stopped on 5853 iteration
3017 [ 0.06146141 -0.68603187  0.85260956  1.21305757] 0.5558187108126926  Stopped on 3017 iteration
2725 [-0.24263494 -0.71350201  1.04144098  1.34998906] 0.5275400371619717  Stopped on 2725

{'alpha': 0.1,
 'early_stop_iteration': 5853,
 'error': 0.28384397239036285,
 'iterations': 6000,
 'weights': array([-8.53579974, -1.24563049, -1.51151549,  7.33020294])}



---



3. Создайте функцию `calc_pred_proba`, возвращающую предсказанную вероятность класса 1 (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией `eval_LR_model`, на выходе - массив `y_pred_proba`).

In [165]:
def calc_pred_proba(X, w):    
    y_pred_proba = sigmoid(np.dot(X_st, w))
    return y_pred_proba > 0.5  # вероятность отнесения объекта к классу "+1"

In [166]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],  # квалификация репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # поступил или нет ученик на специальность Математика

X_st = X.copy()
X_st[:, 2] = standard_scaler(X[:, 2])

w = best.get('weights')
w

positive_class_st = calc_pred_proba(X_st, w)
positive_class = calc_pred_proba(X, w)

positive_class_st, positive_class

(array([False, False,  True, False,  True, False,  True, False, False,
         True]),
 array([False, False,  True, False,  True, False,  True, False, False,
         True]))



---



4. Создайте функцию `calc_pred`, возвращающую предсказанный класс (на вход подаются значения признаков `Х` и веса, которые уже посчитаны функцией `eval_LR_model`, на выходе - массив `y_pred`).

In [167]:
def calc_pred(X, w):
  y_pred = sigmoid(np.dot(X_st, w))    
  return [1 if y > 0.5 else 0 for y in y_pred]

In [168]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],  # квалификация репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # поступил или нет ученик на специальность Математика

w = best.get('weights')
w

y_pred = calc_pred(X, w)
y_pred

[0, 0, 1, 0, 1, 0, 1, 0, 0, 1]



---



5. Посчитайте accuracy, матрицу ошибок, precision и recall, а также F1-score.


In [169]:
import sklearn.metrics
from sklearn.metrics import precision_recall_curve

def calc_pred_score(X, w):
  y_pred = sigmoid(np.dot(X_st, w))    
  return y_pred

print(list(y))
print(y_pred)

[0, 0, 1, 0, 1, 0, 1, 0, 1, 1]
[0, 0, 1, 0, 1, 0, 1, 0, 0, 1]


In [170]:
accuracy= sklearn.metrics.accuracy_score(y, y_pred)
accuracy

0.9

In [171]:
conf_matrix= sklearn.metrics.confusion_matrix(y, y_pred)
conf_matrix

array([[5, 0],
       [1, 4]])

In [172]:
y_scores = calc_pred_score(X, w)
y_scores

array([0.27476844, 0.16882184, 0.9870639 , 0.21716215, 0.71239605,
       0.3407197 , 0.99627936, 0.10005487, 0.30688926, 0.97222404])

In [173]:


precision, recall, thresholds = precision_recall_curve(y, y_scores)
precision

array([0.83333333, 0.8       , 1.        , 1.        , 1.        ,
       1.        , 1.        ])

In [174]:
recall

array([1. , 0.8, 0.8, 0.6, 0.4, 0.2, 0. ])

In [175]:
thresholds

array([0.30688926, 0.3407197 , 0.71239605, 0.97222404, 0.9870639 ,
       0.99627936])

In [176]:
f1_score=sklearn.metrics.f1_score(y, y_pred)
f1_score

0.888888888888889



---



6. Могла ли модель переобучиться? Почему?

In [177]:
# Да, маленькая выборка